In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score 
import matplotlib.pyplot as plt
import re

# CSV 파일에서 데이터 로드
file_path = '20240703_correlation.csv'
df = pd.read_csv(file_path)

# 열 목록
columns = df.columns

# R^2 값을 저장할 행렬 초기화
r2_matrix = pd.DataFrame(index=columns, columns=columns)

# 각 열을 종속 변수로 지정
for dependent_var in columns:
    independent_vars = df.columns.difference([dependent_var])
    
    for var in independent_vars:
        # 결측치 제거
        valid_data = df[[var, dependent_var]].dropna()
        
        X = valid_data[[var]]
        y = valid_data[dependent_var]
        
        if len(valid_data) > 0:  # 유효한 데이터가 있는지 확인
            # 다항식 특징 변환기 생성 (차수 4로 설정)
            poly = PolynomialFeatures(degree=4)
            X_poly = poly.fit_transform(X)
            
            # 모델 훈련
            model = LinearRegression()
            model.fit(X_poly, y)
            
            # 예측
            y_pred = model.predict(X_poly)
            
            # R^2 계산
            r2 = r2_score(y, y_pred)
            r2_matrix.loc[var, dependent_var] = r2

            # 계수 출력
            intercept, coef_linear, coef_quadratic, coef_cubic, coef_quartic = model.intercept_, model.coef_[1], model.coef_[2], model.coef_[3], model.coef_[4]
            print(f'Independent Var: {var}, Dependent Var: {dependent_var}')
            print(f'Intercept: {intercept}, Linear Coeff: {coef_linear}, Quadratic Coeff: {coef_quadratic}, Cubic Coeff: {coef_cubic}, Quartic Coeff: {coef_quartic}')
        else:
            r2_matrix.loc[var, dependent_var] = np.nan  # 유효한 데이터가 없으면 NaN으로 설정

# r2_matrix를 CSV 파일로 저장
output_file_path = '20240710_quartic_r2.csv'  # 저장할 파일 경로
r2_matrix.to_csv(output_file_path)

# R^2 값이 0.9 이상인 모델 찾기
high_r2_models = r2_matrix[r2_matrix >= 0.90].stack().reset_index()
high_r2_models.columns = ['Independent_Var', 'Dependent_Var', 'R2']


Independent Var: 0-0_S1 (eV), Dependent Var: homo (eV)
Intercept: -4.696434708105128, Linear Coeff: 0.17688613776138862, Quadratic Coeff: -2.8620711998421307, Cubic Coeff: 1.7327522577083794, Quartic Coeff: -0.3322640300465906
Independent Var: 0-0_T1 (eV), Dependent Var: homo (eV)
Intercept: -4.947028496549544, Linear Coeff: -2.179329822787491, Quadratic Coeff: 0.778290004626039, Cubic Coeff: -0.47092253555385644, Quartic Coeff: 0.12408910050509658
Independent Var: adiabatic_S1-S0 (kcal/mol), Dependent Var: homo (eV)
Intercept: -4.366467703150507, Linear Coeff: -0.013252333368458996, Quadratic Coeff: -0.004693669417575878, Cubic Coeff: 0.0001282663849372572, Quartic Coeff: -1.0746390925811533e-06
Independent Var: adiabatic_T1-S0 (kcal/mol), Dependent Var: homo (eV)
Intercept: -4.988366448417143, Linear Coeff: -0.09107884787838137, Quadratic Coeff: 0.0012949565754281954, Cubic Coeff: -3.5682992586971805e-05, Quartic Coeff: 4.281503040159951e-07
Independent Var: dipole_moment_norm_S0 (D)

In [ ]:
# 첫 번째 모델에 대해 CSV 파일과 그래프 저장
for index, row in high_r2_models.iterrows():
    independent_var = row['Independent_Var']
    dependent_var = row['Dependent_Var']
    r2 = row['R2']
    
    # 결측치 제거
    valid_data = df[[independent_var, dependent_var]].dropna()
    
    X = valid_data[[independent_var]]
    y = valid_data[dependent_var]
    
    # 다항식 특징 변환기 생성 (차수 4로 설정)
    poly = PolynomialFeatures(degree=4)
    X_poly = poly.fit_transform(X)
    
    # 모델 학습 및 예측
    model = LinearRegression()
    model.fit(X_poly, y)
    y_pred = model.predict(X_poly)
    
    # 실제 값과 예측치 저장
    result_df = pd.DataFrame({
        'Real': y,
        'Predict': y_pred
    })
    
    # 파일 이름에서 유효하지 않은 문자 제거
    def sanitize_filename(filename):
        return re.sub(r'[\\/*?:"<>|]', "_", filename)

    csv_filename = sanitize_filename(f'{independent_var}__{dependent_var}.csv')
    result_df.to_csv(csv_filename, index=False)
    
    # 그래프 생성
    plt.figure()
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k-', lw=2)
    plt.scatter(y, y_pred, color='red', alpha=0.5)
    plt.xlabel(f'Actual values of {dependent_var}')
    plt.ylabel(f'Predicted values of {dependent_var}')
    plt.title(f'Model: {independent_var}__{dependent_var}')

    # R^2 값 그래프에 추가
    plt.text(x=0.05, y=0.95, s=f'$R^2 = {r2:.4f}$', fontsize=12, ha='left', va='top', transform=plt.gca().transAxes)

    # 그래프 이미지 파일로 저장
    img_filename = sanitize_filename(f'{independent_var}__{dependent_var}.png')
    plt.savefig(img_filename)
    plt.close()